The Plotly Community Feed has more than a million user-generated charts, many of which include data.  Conveniently, the precident project vizML has packeged up data from the plotly community feed, along with a package of attributes they calculated from cleaned and de-duplicated data.  As a shortcut, I'll start with their processed attributes. 

The vizML features are available at http://vizml-repository.s3.amazonaws.com/features.tar.gz and can be downloaded, unzipped, and put into the features folder by running the shell script vizML_retrieve_data.sh in the root directory of this repository. 

This notebook saves a data file called "

## Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_dir_name = '""../Data"
output_file_name = "features_with_chart_type_labels_888k.csv"

## Load data

In [3]:
chart_outcomes = pd.read_csv("../features/processed/chart_outcomes.csv")
chart_outcomes.head()

,fid,both_single_srcs,has_single_src,has_single_x_src,has_single_y_src,single_x_src_id,single_y_src_id,single_x_src_order,single_y_src_order,num_x_axes,num_y_axes,num_unique_x_axes,num_unique_y_axes,one_subplot,num_traces,num_trace_types,trace_types,is_all_one_trace_type,all_one_trace_type,num_fields_used_by_data
0,mipi:193,False,False,False,False,NaN,NaN,NaN,NaN,6,6,6,6,False,9,1,"['line', 'line', 'line', 'line', 'line', 'line...",True,line,18
1,maragones:847,False,False,False,False,NaN,NaN,NaN,NaN,0,0,0,0,True,1,1,['bar'],True,bar,2
2,Umbric:106,False,True,True,False,Umbric:107:27b2ef,NaN,0.0,NaN,0,0,0,0,True,3,1,"['bar', 'bar', 'bar']",True,bar,4
3,maragones:844,False,False,False,False,NaN,NaN,NaN,NaN,0,0,0,0,True,1,1,['bar'],True,bar,2
4,dosauto:547,False,True,True,False,dosauto:548:20069f,NaN,0.0,NaN,2,2,2,2,False,6,1,"['line', 'line', 'line', 'line', 'line', 'line']",True,line,7


In [4]:
chart_outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989068 entries, 0 to 1989067
Data columns (total 20 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   fid                      object 
 1   both_single_srcs         bool   
 2   has_single_src           bool   
 3   has_single_x_src         bool   
 4   has_single_y_src         bool   
 5   single_x_src_id          object 
 6   single_y_src_id          object 
 7   single_x_src_order       float64
 8   single_y_src_order       float64
 9   num_x_axes               int64  
 10  num_y_axes               int64  
 11  num_unique_x_axes        int64  
 12  num_unique_y_axes        int64  
 13  one_subplot              bool   
 14  num_traces               int64  
 15  num_trace_types          int64  
 16  trace_types              object 
 17  is_all_one_trace_type    bool   
 18  all_one_trace_type       object 
 19  num_fields_used_by_data  int64  
dtypes: bool(6), float64(2), int64(7), object(5)
me

In [5]:
chart_outcomes.one_subplot.value_counts()

True     1912716
False      76352
Name: one_subplot, dtype: int64

In [6]:
chart_outcomes.num_trace_types.value_counts()

1    1688619
2     271128
0      23270
3       5650
4        399
5          2
Name: num_trace_types, dtype: int64

I want to know what chart types there are. For the 168k+ charts that have only one trace type, what are the most common trace types? This would be equivalent to "chart type" for that group.  

line, scatter, bar, box, pie, histogram, heatmap all seem like things we would want to look at. 
possibly, we'd want to learn sankey plots, as they are common in genomics data. There are 1987 of those. 

Client is intersted in learning cases where data is best represented as a table.  there are 620 of those.

There are tons of cases in this data in which trace types are misspelled or capitalized, but those are not very frequent, and i think we can drop them. 

In [8]:
pd.set_option('display.max_rows', 100)
print(chart_outcomes.all_one_trace_type.value_counts(dropna=True).to_frame().head(15))

                    all_one_trace_type
line                            665965
scatter                         429481
bar                             267919
box                              77367
pie                              60298
histogram                        51749
heatmap                          49835
scatter3d                        30201
surface                          11754
choropleth                       11042
scattergeo                        7424
scattermapbox                     4971
scattergl                         3856
contour                           3308
mesh3d                            2547
sankey                            1987
histogram2d                       1946
candlestick                       1542
area                              1288
scatterternary                     922
timeseries                         732
table                              620
histogram2dcontour                 600
parcoords                          527
ohlc                     

Make a master dataframe of the reduced data set, containing only charts with 1 trace type and trace types that we are interested in:

In [9]:
chart_types = ['line', 'scatter', 'bar', 'box', 'pie', 'histogram', 'heatmap', 'sankey', 'table']
chart_outcomes_r = chart_outcomes[chart_outcomes['all_one_trace_type'].isin(chart_types)]
print(chart_outcomes.shape)
print(chart_outcomes_r.shape)
print(f"selected {chart_outcomes_r.shape[0]} charts, or {chart_outcomes_r.shape[0]/chart_outcomes.shape[0]} of total available charts")

(1989068, 20)
(1605221, 20)
selected 1605221 charts, or 0.8070216805056438 of total available charts


I'll merge this with the features dataset by fid using an inner join, to make sure we have 

In [10]:
chart_type_label = chart_outcomes_r[['fid', 'all_one_trace_type']]
chart_type_label.columns = ['fid', 'labels']

In [11]:
features_df = pd.read_csv("../features/processed/features_aggregate_single_pairwise.csv")
features_df.head()

,fid,exists-agg-num,exists-agg-has,exists-agg-only_one,exists-agg-all,exists-agg-percentage,length-agg-mean,length-agg-var,length-agg-std,length-agg-avg_abs_dev,...,num_shared_words-agg-med_abs_dev,num_shared_words-agg-min,num_shared_words-agg-max,percent_shared_words-agg-mean,percent_shared_words-agg-var,percent_shared_words-agg-std,percent_shared_words-agg-avg_abs_dev,percent_shared_words-agg-med_abs_dev,percent_shared_words-agg-min,percent_shared_words-agg-max
0,automata:52280,0.770345,True,False,False,0.770345,0.067238,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,barisguclu:21,-0.707945,True,False,False,-0.707945,-0.222991,-0.137411,-0.206446,-0.205936,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,glengal:43,-0.707945,True,False,False,-0.707945,-0.225834,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,violetkks:965,2.495016,True,False,False,2.495016,0.377477,9.200560,8.211788,8.235231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,eunyong0718:32,-0.707945,True,False,False,-0.707945,-0.225509,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# merge features with labels with an inner merge, keeping only charts in both data frames
df = features_df.merge(chart_type_label, left_on='fid', right_on='fid', how='inner')
# check the shape of that new df
df.shape
print(f"we have {df.shape[0]} charts with attributes, or {df.shape[0]/chart_outcomes.shape[0]} of total available charts")

we have 888484 charts with attributes, or 0.4466835724067754 of total available charts


In [14]:
duplicates = df[df.duplicated()]
duplicates.shape

(0, 848)

In [22]:
df.head()

,fid,exists-agg-num,exists-agg-has,exists-agg-only_one,exists-agg-all,exists-agg-percentage,length-agg-mean,length-agg-var,length-agg-std,length-agg-avg_abs_dev,...,num_shared_words-agg-min,num_shared_words-agg-max,percent_shared_words-agg-mean,percent_shared_words-agg-var,percent_shared_words-agg-std,percent_shared_words-agg-avg_abs_dev,percent_shared_words-agg-med_abs_dev,percent_shared_words-agg-min,percent_shared_words-agg-max,labels
0,automata:52280,0.770345,True,False,False,0.770345,0.067238,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,scatter
1,barisguclu:21,-0.707945,True,False,False,-0.707945,-0.222991,-0.137411,-0.206446,-0.205936,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,scatter
2,glengal:43,-0.707945,True,False,False,-0.707945,-0.225834,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,bar
3,eunyong0718:32,-0.707945,True,False,False,-0.707945,-0.225509,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,scatter
4,likunyi2:24,2.002253,True,False,False,2.002253,-0.223560,-0.137411,-0.207664,-0.207261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,scatter


In [29]:
df.iloc[1,1:]

exists-agg-num                         -0.707945
exists-agg-has                              True
exists-agg-only_one                        False
exists-agg-all                             False
exists-agg-percentage                  -0.707945
                                          ...   
percent_shared_words-agg-avg_abs_dev           0
percent_shared_words-agg-med_abs_dev           0
percent_shared_words-agg-min                   0
percent_shared_words-agg-max                   0
labels                                   scatter
Name: 1, Length: 847, dtype: object

In [18]:
# save a copy of this
# df.to_csv('features_with_chart_type_labels_888k.csv')

output_file_name = os.path.join(data_dir_name, output_file_name)
df.to_csv(output_file_name)

print("saved to ", data_dir_name)